In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

Mounted at /content/drive


In [2]:
!pip install jsonlines datasets git+https://github.com/google-research/bleurt.git evaluate sacrebleu

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-y0yj2o08
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-y0yj2o08
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


In [3]:
import jsonlines
data_PATH = drive_PATH + '/data-main/exp2.4.tren-n128-t1-e0.02.jsonl'
data = []

with jsonlines.open(data_PATH) as f:
    for line in f.iter():
        data.append(line)

In [4]:
import tqdm
import jsonlines
import sys
import numpy as np

# Eval - BLEURT
from datasets import load_metric
bleurt = load_metric('bleurt', 'BLEURT-20-D3')

<ipython-input-4-3f117e7ccb37>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleurt = load_metric('bleurt', 'BLEURT-20-D3')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bleurt contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bleurt/bleurt.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [5]:
def BLEURT_mbrd(candidates):
  k_samples = len(candidates)
  score_matrix = np.zeros((k_samples, k_samples))

  for j1 in range(k_samples):
    references=[candidates[j1][0] for j2 in range(k_samples)]
    predictions=[candidates[j2][0] for j2 in range(k_samples)]
    scores = bleurt.compute(predictions=predictions, references=references)['scores']
    score_matrix[ : ,j1] = scores

  # remove votes for self
  for j1 in range(k_samples):
    for j2 in range(k_samples):
      if j1 == j2:
        score_matrix[j2,j1] = 0

  sum_scores = np.sum(score_matrix, axis=1)
  index = np.argmax(sum_scores)
  mbr_winner = candidates[index]

  return mbr_winner, index, sum_scores, score_matrix

In [6]:
dataset_name = 'TREN'
num_examples = 1000
assert(num_examples == len(data))
num_candidates = 128
dataset = data
for j in tqdm.tqdm(range(num_examples)):
    predictions=[dataset[j]['candidates'][i][0] for i in range(num_candidates)]
    references=[dataset[j]['ref'] for i in range(num_candidates)]
    dataset[j]['BLEURT_cand_scores'] = np.array(bleurt.compute(predictions=predictions, references=references)['scores'], dtype=float).tolist()
    # MBR BLEURT
    mbr_winner, index, sum_scores, score_matrix = BLEURT_mbrd(dataset[j]['candidates'])
    dataset[j]['MBR_BLEURT_winner'] = mbr_winner
    dataset[j]['MBR_BLEURT_winner_index'] = int(index)
    dataset[j]['MBR_BLEURT_cand_scores'] = sum_scores.tolist()
    dataset[j]['MBR_BLEURT_score_matrix'] = score_matrix.tolist()

bleurt_dataset_PATH = drive_PATH+ f'/data-main/bleurt-{dataset_name}.jsonl'
with jsonlines.open(bleurt_dataset_PATH, mode='w') as writer:
    for item in dataset:
        writer.write(item)

100%|██████████| 1000/1000 [12:40:49<00:00, 45.65s/it]
